前面总结了RNN模型，由于RNN依然存在长序列是梯度消失问题，因此解决梯度消失问题的LSTM被发明出来,引用牛人的一句话：

Humans don’t start their thinking from scratch every second. As you read this essay, you understand each word based on your understanding of previous words. You don’t throw everything away and start thinking from scratch again. Your thoughts have persistence.

## 1.RNN到LSTM

在RNN模型里，每个序列索引位置t都有一个隐藏状态$h^{(t)}$

![RNN-01](./pics/RNN-01.png)

如果省略每层都有的$o^{(t)}$,$L^{(t)}$,$y^{(t)}$，则RNN模型可以简化成如下：

![RNN](./pics/RNN-02.png)

图中可以很清晰看出在隐藏状态$h^{(t)}$由$x^{(t)}$和$h^{(t-1)}$得到。得到$h^{(t)}$后一方面用户当前的模型损失计算，另一方面
用于计算下一层$h^{(t+1)}$

由于RNN梯度消失的问题，大牛们对于序列索引位置t的隐藏结构做了改进，可以说通过一些技巧让隐藏结构复杂了起来，来避免梯度消失的问题，这样的特殊RNN就是我们的LSTM。由于LSTM有很多的变种，这里我们以最常见的LSTM为例讲述。LSTM的结构如下图:

![ls1](./pics/LSTM3.png)

![ls](./pics/LSTM3notation.png)

## 2.LSTM模型结果剖析

上面我们给出了LSTM的模型结构，下面我们就一点点的剖析LSTM模型在每个序列索引位置t时刻的内部结构
从上图中可以看出，在每个序列索引位置t时刻向前传播的除了和RNN一样的隐藏状态$h^{(t)}$还多了另一个隐藏状态
如下图中$C_{t-1}$到$C_{t}$的线，这个隐藏状态一般称为细胞状态（Cell State）记为$C^{(t)}$

![ls-1](./pics/LSTM3-Ct-1-to-C.png)

除细胞状态，LSTM图还有很多奇怪的结构，这些结构一般称作为门结构，LSTM在每个序列索引位置t的门
一般包括遗忘门，输入门，输出门三种

## 3.Step-by-Step LSTM Walk Through

#### 3.1 遗忘门

LSTM的第一步是决定我们在细胞状态（Cell State）中丢弃什么，这是通过激活函数sigmoid 得到的“称为遗忘门”决定，它作用在$h_{t-1}$和$x_{t}$上，为$C_{t-1}$输出0或1的数，1表示保留，0则去掉。$f_{(t)}$代表了遗忘上一层隐藏细胞状态的概率，其中$W_{f}$，$U_{f}$，$b_{f}$为线性关系的系数和偏置，$\sigma$为sigmoid激活函数。

Let’s go back to our example of a language model trying to predict the next word based on all the previous ones. In such a problem, the cell state might include the gender of the present subject, so that the correct pronouns can be used. When we see a new subject, we want to forget the gender of the old subject.

![fortget](./pics/LSTM3-focus1-f.png)

$$
\begin{aligned}
f^{(t)} = \sigma(W_{f}h^{(t-1)} + U_{f}x^{(t)} + b_{f})
\end{aligned}
$$

#### 3.2 输入门

这一步将决定我们将在细胞状态中存储什么，这里有两个部分，首先通过一个sigmoid层称为“输入门”决定我们将更新哪些值
然后，通过一个tanh层创建一个可以加入细胞状态的的候选值的向量$\hat C_{t}$,最后我们将这两个结合起来去更新细胞状态；

在这个语言模型中，我们想添加新样本的gender到细胞状态，去替换我们遗忘的旧样本的gender


In the case of the language model, this is where we’d actually drop the information about the old subject’s gender and add the new information, as we decided in the previous steps.
![](./pics/LSTM3-focus2-i.png)

$$
\begin{aligned}
i^{(t)} = \sigma(W_{i}h^{(t-1)} + U_{i}x^{(t)} + b_{i})
\end{aligned}
$$

$$
\begin{aligned}
\hat C^{(t)} = \tanh(W_{\hat C}h^{(t-1)} + U_{\hat C}x^{(t)} + b_{\hat C})
\end{aligned}
$$

是时候更新旧的细胞状态$C_{t-1}$到新的细胞状态$C_{t}$了，首先用$f_{t}$乘以$C_{t_1}$，遗忘掉决定要遗忘的，然后加上$i_{t}\odot\hat C_{t}$，
![](./pics/LSTM3-focus3-i-2.png)
其中$\ast$在这里表示为Hadamard积，即相同维度的矩阵，对应位置元素相乘

$$
\begin{aligned}
C^{(t)} =C^{(t-1)}\ast f^{(t)} + i^{(t)}\odot \hat C^{(t)}
\end{aligned}
$$
 

#### 3.3 输出门

![](./pics/LSTM3-focus-o4.png)

从图中可以看出，隐藏状态$h_{t}$的更新由两部分组成，第一部分是$o_{t}$,它由上一序列的隐藏状态$h_{t-1}$
和本序列数据$x_{t}$,通过激活函数sigmoid得到，第二部分由隐藏状态$C_{t}$和tanh激活函数组成，然后两部分做Hadamard乘积

$$
\begin{aligned}
o^{(t)} = \sigma(W_{o}h^{(t-1)} + U_{i}x^{(t)} + b_{o})
\end{aligned}
$$

$$
\begin{aligned}
h^{(t)} = o^{(t)}\odot tanh(C^{(t)})
\end{aligned}
$$

## 4.LSTM前向传播算法

（算法中我们统一将t下标改为上标）LSTM模型有两个隐藏状态$h^{(t)}$,$C^{(t)}$，模型参数几乎是RNN的4倍，
因为现在多了$W_{f},U_{f},b_{f},W_{\hat c},U_{\hat c},b_{\hat c},W_{i},U_{i},b_{i},W_{o},U_{o},b_{o}$这些参数

前向传播过程在每个序列索引位置的过程为：

1)更新遗忘门输出：
$$
\begin{aligned}
f^{(t)} = \sigma(W_{f}h^{(t-1)} + U_{f}x^{(t)} + b_{f}) ……式（4.1）
\end{aligned}
$$

2)更新输入门两部分输出：
$$
\begin{aligned}
i^{(t)} = \sigma(W_{i}h^{(t-1)} + U_{i}x^{(t)} + b_{i}) ……式（4.21）
\end{aligned}
$$

$$
\begin{aligned}
\hat C^{(t)} = \tanh(W_{\hat C}h^{(t-1)} + U_{\hat C}x^{(t)} + b_{\hat C}) ……式（4.22）
\end{aligned}
$$

3）更新细胞状态：
$$
\begin{aligned}
C^{(t)} =C^{(t-1)}\odot f^{(t)} + i^{(t)}\odot \hat C^{(t)} ……式（4.3）
\end{aligned}
$$

4)更新输出门输出：
$$
\begin{aligned}
o^{(t)} = \sigma(W_{o}h^{(t-1)} + U_{i}x^{(t)} + b_{o}) ……式（4.41）
\end{aligned}
$$

$$
\begin{aligned}
h^{(t)} = o^{(t)}\odot tanh(C^{(t)}) ……式（4.42）
\end{aligned}
$$

5）更新当前序列索引预测输出：
$$
\begin{aligned}
\hat y^{(t)} = \sigma(Vh^{(t)} + c) ……式（4.5）
\end{aligned}
$$

## 5.LSTM反向传播

有了LSTM前向传播算法，推导反向传播算法就容易了，思路和RNN的反向传播算法思路一致，也是通过梯度下降
法迭代更新所有的参数，关键点在于计算所有参数基于损失函数的偏导数。

在RNN中，为了反向传播误差，通过隐藏状态$h^{(t)}$的梯度一步步向前传播，在LSTM也是类似，只不过这里有两个隐藏状态
$h^{(t)}$和$C^{(t)}$，这里定义两个$\delta$，即：

$$
\begin{aligned}
\delta_{h}^{(t)} = \frac{\partial{Lost}}{\partial{h^{(t)}}}……式（5.1）
\end{aligned}
$$

$$
\begin{aligned}
\delta_{C}^{(t)}= \frac{\partial{Lost}}{\partial{C^{(t)}}}……式（5.2）
\end{aligned}             
$$

反向传播时只使用了$\delta_{C}^{(t)}$,变量$\delta_{h}^{(t)}$仅为帮助我们在某一层计算用，并没有参数反向传播，这里要注意,如果没懂，请看输出门
![](./pics/unrolled_backward.png)

而在最后的序列索引位置$\tau$的$\delta_{h}^{(\tau)}$和$\delta_{C}^{(\tau)}$为：

$$
\begin{aligned}
\delta_{h}^{(t)} =&\frac{\partial{Lost}}{\partial{O^{(t)}}}\frac{\partial{O}^{(\tau)}}{\partial{h}^{(\tau)}}\\
=&V^{T}(\hat {y}^{(\tau)} - y^{(\tau)})  ……式（5.3）
\end{aligned}
$$

$$
\begin{aligned}
\delta_{C}^{(t)}=&\frac{\partial{Lost}}{\partial{C^{(t)}}}\frac{\partial{h}^{(\tau)}}{\partial{C}^{(\tau)}}\\
=&\delta_{h}^{(\tau)}\odot {O}^{(\tau)}\odot(1-\tanh^2(C^{(\tau)}))  ……式（5.4）
\end{aligned}
$$

接着我们由$\delta_{C}^{(t+1)}$反向推导$\delta_{C}^{(t)}$。
$\delta_{h}^{(t)}$的梯度由本层的输出梯度误差决定，即：

$$
\begin{aligned}
\delta_{h}^{(t)} =&\frac{\partial{Lost}}{\partial{h^{(t)}}}\\
=&V^{T}(\hat {y}^{(t)} - y^{(t)})  ……式（5.5）
\end{aligned}
$$

而$\delta_{C}^{(t)}$的反向传播误差由前一层$\delta_{C}^{(t+1)}$的梯度误差和本层的从$h^{(t)}$传回来的梯度误差两部分组成，即：

$$
\begin{aligned}
\delta_{C}^{(t)}=&\frac{\partial{Lost}}{\partial{C^{(t+1)}}}\frac{\partial{C}^{(t+1)}}{\partial{C}^{(C)}}+\frac{\partial{Lost}}{\partial{h^{(t)}}}\frac{\partial{h^{(t)}}}{\partial{C^{(t)}}}\\
=&\delta_{C}^{(t+1)}\odot f^{(t+1)} + \delta_{h}^{(t)}\odot {O}^{(t)}\odot(1-\tanh^2(C^{(t)})) ……式（5.6）f^{(t+1)}参见公式4.42
\end{aligned}
$$

有了$\delta_{h}^{(t)}$和$\delta_{C}^{(t)}$，计算$W_{f},U_{f},b_{f},W_{\hat c},U_{\hat c},b_{\hat c},W_{i},U_{i},b_{i},W_{o},U_{o},b_{o}$就容易:

$W_{f}$的梯度计算：

$$
\begin{aligned}
\frac{\partial{Lost}}{\partial{W_{f}}} =&\frac{\partial{Lost}}{\partial{C^{(t)}}}\frac{\partial{C^{(t)}}}{\partial{f^{(t)}}}\frac{\partial{f^{(t)}}}{\partial{W_{f}}}\\
=&\sum_{t=1}^{\tau}\delta_{C}^{(t)}\odot {C^{(t-1)}}\odot{f^{(t)}}\odot{(1-f^{(t)})}(h^{(t-1)})^T
\end{aligned}
$$

$U_{f}$梯度的计算：

$$
\begin{aligned}
\frac{\partial{Lost}}{\partial{U_{f}}} =&\frac{\partial{Lost}}{\partial{C^{(t)}}}\frac{\partial{C^{(t)}}}{\partial{f^{(t)}}}\frac{\partial{f^{(t)}}}{\partial{U_{f}}}\\
=&\sum_{t=1}^{\tau}\delta_{C}^{(t)}\odot {C^{(t-1)}}\odot{f^{(t)}}\odot{(1-f^{(t)})}(x^{(t)})^T
\end{aligned}
$$

$b_{f}$梯度的计算：

$$
\begin{aligned}
\frac{\partial{Lost}}{\partial{b_{f}}} =&\frac{\partial{Lost}}{\partial{C^{(t)}}}\frac{\partial{C^{(t)}}}{\partial{f^{(t)}}}\frac{\partial{f^{(t)}}}{\partial{b_{f}}}\\
=&\sum_{t=1}^{\tau}\delta_{C}^{(t)}\odot {C^{(t-1)}}\odot{f^{(t)}}\odot{(1-f^{(t)})}
\end{aligned}
$$

参考：
http://colah.github.io/posts/2015-08-Understanding-LSTMs/
https://www.cnblogs.com/pinard/p/6519110.html

In [ ]:
basicLSTMCell源码分析：
def call(self, inputs, state):
    """Long short-term memory cell (LSTM).

    Args:
      inputs: `2-D` tensor with shape `[batch_size, input_size]`.
      state: An `LSTMStateTuple` of state tensors, each shaped
        `[batch_size, self.state_size]`, if `state_is_tuple` has been set to
        `True`.  Otherwise, a `Tensor` shaped
        `[batch_size, 2 * self.state_size]`.

    Returns:
      A pair containing the new hidden state, and the new state (either a
        `LSTMStateTuple` or a concatenated state, depending on
        `state_is_tuple`).
    """
    sigmoid = math_ops.sigmoid
    one = constant_op.constant(1, dtype=dtypes.int32)
    # Parameters of gates are concatenated into one multiply for efficiency.
    if self._state_is_tuple:
      c, h = state #?,256
    else:
      c, h = array_ops.split(value=state, num_or_size_splits=2, axis=one)
    #将输入和上一个隐层按第二维度拼接，得到（？，step+hidden_size）， 
    gate_inputs = math_ops.matmul(
        array_ops.concat([inputs, h], 1), self._kernel) #input(?,28) h (?,256) kernel(284,1024) 
    gate_inputs = nn_ops.bias_add(gate_inputs, self._bias) #(?,1024)
    
    #将数据分割为四等分每个（？，256）
    # i = input_gate, j = new_input, f = forget_gate, o = output_gate
    i, j, f, o = array_ops.split( #(?,256)
        value=gate_inputs, num_or_size_splits=4, axis=one)

    forget_bias_tensor = constant_op.constant(self._forget_bias, dtype=f.dtype)
    # Note that using `add` and `multiply` instead of `+` and `*` gives a
    # performance improvement. So using those at the cost of readability.
    add = math_ops.add
    multiply = math_ops.multiply
    new_c = add(multiply(c, sigmoid(add(f, forget_bias_tensor))),#(?,256)
                multiply(sigmoid(i), self._activation(j)))
    new_h = multiply(self._activation(new_c), sigmoid(o)) #矩阵对应元素相乘(?,256)

    if self._state_is_tuple:
      new_state = LSTMStateTuple(new_c, new_h)#(?,256)
    else:
      new_state = array_ops.concat([new_c, new_h], 1)
    return new_h, new_state
